In [ ]:
from ultralytics import YOLO
import cv2
import csv
import numpy as np
from sort.sort import *

# Your custom utility functions
from util import get_car, read_license_plate, write_csv

# Initialize variables
results = {}
mot_tracker = Sort()
coco_model = YOLO('yolov8m.pt')
license_plate_detector = YOLO('best.pt')

# Change this line to use the default camera
cap = cv2.VideoCapture('IMG_9518.jpeg')

# Initialize CSV writer
csv_file = open('output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame_nmr', 'car_id', 'car_bbox', 'license_bbox', 'license_text', 'license_bbox_score', 'license_text_score'])

# Main loop
vehicles = [2, 3, 5, 7]
frame_nmr = -1
ret = True

while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        #frame = cv2.resize(frame, (1280,720))
        results[frame_nmr] = {}
        detections = coco_model(frame)[0]
        detections_ = [[x1, y1, x2, y2, score] for x1, y1, x2, y2, score, class_id in detections.boxes.data.tolist() if int(class_id) in vehicles]
        track_ids = mot_tracker.update(np.array(detections_))
        license_plates = license_plate_detector(frame)[0]
        
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)
            if car_id != -1:
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop)
                print('License Plate:',license_plate_text,license_plate_text_score)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
                    
                    # Write to CSV
                    csv_writer.writerow([frame_nmr, car_id, [xcar1, ycar1, xcar2, ycar2], [x1, y1, x2, y2], license_plate_text, score, license_plate_text_score])
                    # Draw bounding boxes and text on the frame
                    cv2.rectangle(frame, (int(xcar1), int(ycar1)), (int(xcar2), int(ycar2)), (0, 255, 0), 2)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                    cv2.putText(frame, license_plate_text, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Show the frame in a window
        cv2.imshow('Real-time License Plate Recognition', frame)
        # Press 'q' to quit
        if cv2.waitKey(1000) & 0xFF == ord('q'):
            break




# Release video and close CSV
cap.release()
csv_file.close()
cv2.destroyAllWindows()


Using CPU. Note: This module is much faster with a GPU.



0: 480x640 1 car, 392.2ms
Speed: 4.7ms preprocess, 392.2ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 707.7ms
Speed: 3.6ms preprocess, 707.7ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
License Plate: LUD 0.9979056583209936


2025-02-16 19:06:55.887 Python[39253:3162507] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-16 19:06:55.887 Python[39253:3162507] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


: 

In [1]:
!pip3 install ultralytics.utils


^C


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from sort.sort import *

# Your custom utility functions
from util import get_car, read_license_plate

# Initialize models
coco_model = YOLO('yolov8m.pt')  # Vehicle detection model
license_plate_detector = YOLO('best.pt')  # License plate detection model

# Load the image
image_path = 'IMG_9518.jpeg'  # Path to the input image
frame = cv2.imread(image_path)

# Initialize SORT tracker
mot_tracker = Sort()

# Process the image
vehicles = [2, 3, 5, 7]  # Vehicle class IDs in COCO dataset
results = {}
frame_nmr = 0
results[frame_nmr] = {}

detections = coco_model(frame)[0]
detections_ = [[x1, y1, x2, y2, score] for x1, y1, x2, y2, score, class_id in detections.boxes.data.tolist() if int(class_id) in vehicles]
track_ids = mot_tracker.update(np.array(detections_))
license_plates = license_plate_detector(frame)[0]

for license_plate in license_plates.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = license_plate
    xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)
    if car_id != -1:
        license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
        license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop)
        
        if license_plate_text is not None:
            results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                          'license_plate': {'bbox': [x1, y1, x2, y2],
                                                            'text': license_plate_text,
                                                            'bbox_score': score,
                                                            'text_score': license_plate_text_score}}
            
            # Draw bounding boxes and text on the frame
            cv2.rectangle(frame, (int(xcar1), int(ycar1)), (int(xcar2), int(ycar2)), (0, 255, 0), 2)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            cv2.putText(frame, license_plate_text, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

# Save the processed image
output_path = 'output_image.jpg'
cv2.imwrite(output_path, frame)
cv2.imshow('License Plate Detection', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f'Processed image saved as {output_path}')


Using CPU. Note: This module is much faster with a GPU.



0: 480x640 1 car, 391.7ms
Speed: 4.2ms preprocess, 391.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 769.1ms
Speed: 15.6ms preprocess, 769.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
xxxx [([[39, 12], [462, 12], [462, 231], [39, 231]], '1229', 0.999042088857666), ([[514, 37], [813, 37], [813, 187], [514, 187]], 'LUD', 0.9977579329041749)]


2025-02-16 18:35:55.748 Python[38545:3135626] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-16 18:35:55.748 Python[38545:3135626] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # Load an official YOLO model to check if it works
model(frame)[0]

ModuleNotFoundError: import of ultralytics.yolo.utils.callbacks.clearml halted; None in sys.modules

In [ ]:
import sys
sys.modules['ultralytics.yolo.utils.callbacks.clearml'] = None

from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # Load YOLO model
results = model.predict('IMG.jpeg', save=True, verbose=True)  # Run inference


ModuleNotFoundError: import of ultralytics.yolo.utils.callbacks.clearml halted; None in sys.modules